In [ ]:
import pandas as pd
import numpy as np
import plotly
import plotly.graph_objs as go
plotly.offline.init_notebook_mode()
import os
import random

DATA_DIR = "data/"
SELECTED_DATA_DIR = "selected-data/"
CSV_DIR = DATA_DIR + "csv/"

In [ ]:
movies = pd.read_csv(SELECTED_DATA_DIR + "best_movie_ratings_features_engineered.csv", index_col=0)
movies = movies.set_index("title")
movies.sample()

In [ ]:
users = pd.DataFrame()

LIMIT=100

files = os.listdir(SELECTED_DATA_DIR + "csv/")
random.shuffle(files)

for (i, filename) in enumerate(files):
    frame = pd.read_csv(SELECTED_DATA_DIR + "csv/" + filename, header=-1)
    users = users.append(frame, ignore_index=True)
    if (i > LIMIT):
        break
    
users.columns = ["user", "movie", "rating", "link"]
users.rating = users.rating/10
users.sample()

In [ ]:
user = users[users.user == users.sample().user.item()]
user = user[user.movie.isin(movies.index)] # get only recommandations for movies that are in our list
for id, recomm in user.iterrows():
    assert(recomm.movie in movies.index)
user = user.set_index("movie")
user.sample()

In [ ]:
movies_user = movies[movies.index.isin(user.index)]
movies_user.sample()

In [ ]:
s = 1
def utility(user_features, movie_features):
    return user_features.dot(movie_features)

def get_movie_features(movie):
    if isinstance(movie, pd.Series):
        return movie[2:]
    elif isinstance(movie, pd.DataFrame):
        return get_movie_features(movie.loc[movie.index[0]])
    else:
        raise TypeError("{} should be a Series or DataFrame".format(movie))
    

def greedy_choice(user_features, movies, epsilon=0.1):
    if random.random() > epsilon: # choose the best
        utilities = np.zeros(movies.shape[0])
        for i, (title, movie) in enumerate(movies.iterrows()):
            movie_features = get_movie_features(movie)
            utilities[i] = utility(user_features, movie_features)
        return movies[movies.index == movies.index[utilities.argmax()]]
    else:
        return movies.sample()
        
def iterative_mean(old, new, t):
    return ((t-1) / t) * old + (1/t) * new
    
def update_features(user_features, movie_features, rating, t):
    return iterative_mean(user_features, movie_features * rating, t)

user_features = np.zeros(movies_user.shape[1] - 2)
Ui = 0

for t in range(1, 10000):
    recommandation = greedy_choice(user_features, movies_user)
    recommandation_features = get_movie_features(recommandation)
    user_rating = user.get_value(recommandation.index[0], "rating")
    user_features = update_features(user_features, recommandation_features, user_rating, t)
print(user_features)